# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d18/93c648398ff85fad51308f4ff8d11c2e8d8e66392462ffe79f3fb628.csv`. The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Shanghai, Shanghai, China**, and the stations the data comes from are shown on the map below.

In [1]:
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd

def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('BinSize_d{}.csv'.format(binsize))
    
    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()


leaflet_plot_stations(18,'93c648398ff85fad51308f4ff8d11c2e8d8e66392462ffe79f3fb628')

In [2]:
df1 = pd.read_csv('93c648398ff85fad51308f4ff8d11c2e8d8e66392462ffe79f3fb628.csv')
df2 = pd.read_csv('BinSize_d18.csv')
df1.head()


,ID,Date,Element,Data_Value
0,JA000047899,2007-12-03,TMAX,158
1,JA000047936,2013-11-12,TMAX,215
2,JA000047675,2011-11-25,TMIN,75
3,JA000047662,2006-08-27,TMAX,272
4,JA000047604,2010-08-22,TMAX,330


In [3]:
df2.head()


,ID,LATITUDE,LONGITUDE,ELEVATION,STATE,NAME,GSNFLAG,HCNFLAG,WMOID,x,y,x_group,y_group,xy_group,hash
0,AE000041196,25.333,55.517,34.0,NaN,SHARJAH INTER. AIRP,GSN,NaN,41196.0,5.579578e+06,2.816905e+06,4567526.494270399 to 6730491.576856308,2404561.4116844907 to 4567526.494270399,"4567526.494270399 to 6730491.576856308, 240456...",624c7824d24a7d4a6321e681e4ceea0631d87459b1d81f...
1,AEM00041194,25.255,55.364,10.4,NaN,DUBAI INTL,NaN,NaN,41194.0,5.567782e+06,2.808232e+06,4567526.494270399 to 6730491.576856308,2404561.4116844907 to 4567526.494270399,"4567526.494270399 to 6730491.576856308, 240456...",624c7824d24a7d4a6321e681e4ceea0631d87459b1d81f...
2,AEM00041217,24.433,54.651,26.8,NaN,ABU DHABI INTL,NaN,NaN,41217.0,5.532707e+06,2.716829e+06,4567526.494270399 to 6730491.576856308,2404561.4116844907 to 4567526.494270399,"4567526.494270399 to 6730491.576856308, 240456...",624c7824d24a7d4a6321e681e4ceea0631d87459b1d81f...
3,AEM00041218,24.262,55.609,264.9,NaN,AL AIN INTL,NaN,NaN,41218.0,5.637301e+06,2.697815e+06,4567526.494270399 to 6730491.576856308,2404561.4116844907 to 4567526.494270399,"4567526.494270399 to 6730491.576856308, 240456...",624c7824d24a7d4a6321e681e4ceea0631d87459b1d81f...
4,AFM00040990,31.500,65.850,1010.0,NaN,KANDAHAR AIRPORT,NaN,NaN,40990.0,6.243199e+06,3.502645e+06,4567526.494270399 to 6730491.576856308,2404561.4116844907 to 4567526.494270399,"4567526.494270399 to 6730491.576856308, 240456...",624c7824d24a7d4a6321e681e4ceea0631d87459b1d81f...


In [4]:
df3 = df2[df2['hash'] == '93c648398ff85fad51308f4ff8d11c2e8d8e66392462ffe79f3fb628']
df3.head()

,ID,LATITUDE,LONGITUDE,ELEVATION,STATE,NAME,GSNFLAG,HCNFLAG,WMOID,x,y,x_group,y_group,xy_group,hash
3665,CHM00057679,28.200,113.083,46.0,NaN,CHANGSHA,NaN,NaN,57679.0,1.108175e+07,3.135701e+06,11056421.742028125 to 13219386.824614033,2404561.4116844907 to 4567526.494270399,"11056421.742028125 to 13219386.824614033, 2404...",93c648398ff85fad51308f4ff8d11c2e8d8e66392462ff...
3666,CHM00057687,28.233,112.867,68.0,NaN,CHANGSHA,NaN,NaN,57687.0,1.105717e+07,3.139371e+06,11056421.742028125 to 13219386.824614033,2404561.4116844907 to 4567526.494270399,"11056421.742028125 to 13219386.824614033, 2404...",93c648398ff85fad51308f4ff8d11c2e8d8e66392462ff...
3670,CHM00057799,27.117,114.967,78.0,NaN,JIAN,NaN,NaN,57799.0,1.137854e+07,3.015277e+06,11056421.742028125 to 13219386.824614033,2404561.4116844907 to 4567526.494270399,"11056421.742028125 to 13219386.824614033, 2404...",93c648398ff85fad51308f4ff8d11c2e8d8e66392462ff...
3672,CHM00057866,26.233,111.617,174.0,NaN,LINGLING,NaN,NaN,57866.0,1.113295e+07,2.916981e+06,11056421.742028125 to 13219386.824614033,2404561.4116844907 to 4567526.494270399,"11056421.742028125 to 13219386.824614033, 2404...",93c648398ff85fad51308f4ff8d11c2e8d8e66392462ff...
3674,CHM00057957,25.333,110.300,174.0,NaN,GUILIN,NaN,NaN,57957.0,1.108539e+07,2.816905e+06,11056421.742028125 to 13219386.824614033,2404561.4116844907 to 4567526.494270399,"11056421.742028125 to 13219386.824614033, 2404...",93c648398ff85fad51308f4ff8d11c2e8d8e66392462ff...


In [5]:
df_total = pd.merge(left=df1,right=df2,how="inner", left_on='ID', right_on="ID")
df_total

,ID,Date,Element,Data_Value,LATITUDE,LONGITUDE,ELEVATION,STATE,NAME,GSNFLAG,HCNFLAG,WMOID,x,y,x_group,y_group,xy_group,hash
0,JA000047899,2007-12-03,TMAX,158,33.250,134.183,186.0,NaN,MUROTOMISAKI,NaN,NaN,47899.0,1.247780e+07,3.697237e+06,11056421.742028125 to 13219386.824614033,2404561.4116844907 to 4567526.494270399,"11056421.742028125 to 13219386.824614033, 2404...",93c648398ff85fad51308f4ff8d11c2e8d8e66392462ff...
1,JA000047899,2006-09-01,TMIN,223,33.250,134.183,186.0,NaN,MUROTOMISAKI,NaN,NaN,47899.0,1.247780e+07,3.697237e+06,11056421.742028125 to 13219386.824614033,2404561.4116844907 to 4567526.494270399,"11056421.742028125 to 13219386.824614033, 2404...",93c648398ff85fad51308f4ff8d11c2e8d8e66392462ff...
2,JA000047899,2014-01-17,TMAX,108,33.250,134.183,186.0,NaN,MUROTOMISAKI,NaN,NaN,47899.0,1.247780e+07,3.697237e+06,11056421.742028125 to 13219386.824614033,2404561.4116844907 to 4567526.494270399,"11056421.742028125 to 13219386.824614033, 2404...",93c648398ff85fad51308f4ff8d11c2e8d8e66392462ff...
3,JA000047899,2008-04-15,TMIN,123,33.250,134.183,186.0,NaN,MUROTOMISAKI,NaN,NaN,47899.0,1.247780e+07,3.697237e+06,11056421.742028125 to 13219386.824614033,2404561.4116844907 to 4567526.494270399,"11056421.742028125 to 13219386.824614033, 2404...",93c648398ff85fad51308f4ff8d11c2e8d8e66392462ff...
4,JA000047899,2015-03-01,TMAX,124,33.250,134.183,186.0,NaN,MUROTOMISAKI,NaN,NaN,47899.0,1.247780e+07,3.697237e+06,11056421.742028125 to 13219386.824614033,2404561.4116844907 to 4567526.494270399,"11056421.742028125 to 13219386.824614033, 2404...",93c648398ff85fad51308f4ff8d11c2e8d8e66392462ff...
5,JA000047899,2006-09-01,TMAX,267,33.250,134.183,186.0,NaN,MUROTOMISAKI,NaN,NaN,47899.0,1.247780e+07,3.697237e+06,11056421.742028125 to 13219386.824614033,2404561.4116844907 to 4567526.494270399,"11056421.742028125 to 13219386.824614033, 2404...",93c648398ff85fad51308f4ff8d11c2e8d8e66392462ff...
6,JA000047899,2014-04-21,TMAX,173,33.250,134.183,186.0,NaN,MUROTOMISAKI,NaN,NaN,47899.0,1.247780e+07,3.697237e+06,11056421.742028125 to 13219386.824614033,2404561.4116844907 to 4567526.494270399,"11056421.742028125 to 13219386.824614033, 2404...",93c648398ff85fad51308f4ff8d11c2e8d8e66392462ff...
7,JA000047899,2014-04-21,TMIN,124,33.250,134.183,186.0,NaN,MUROTOMISAKI,NaN,NaN,47899.0,1.247780e+07,3.697237e+06,11056421.742028125 to 13219386.824614033,2404561.4116844907 to 4567526.494270399,"11056421.742028125 to 13219386.824614033, 2404...",93c648398ff85fad51308f4ff8d11c2e8d8e66392462ff...
8,JA000047899,2012-11-09,TMAX,196,33.250,134.183,186.0,NaN,MUROTOMISAKI,NaN,NaN,47899.0,1.247780e+07,3.697237e+06,11056421.742028125 to 13219386.824614033,2404561.4116844907 to 4567526.494270399,"11056421.742028125 to 13219386.824614033, 2404...",93c648398ff85fad51308f4ff8d11c2e8d8e66392462ff...
9,JA000047899,2006-05-27,TMIN,150,33.250,134.183,186.0,NaN,MUROTOMISAKI,NaN,NaN,47899.0,1.247780e+07,3.697237e+06,11056421.742028125 to 13219386.824614033,2404561.4116844907 to 4567526.494270399,"11056421.742028125 to 13219386.824614033, 2404...",93c648398ff85fad51308f4ff8d11c2e8d8e66392462ff...


In [6]:
df_total = df_total[['ID','Date','Element', 'Data_Value','NAME']]
df_total

,ID,Date,Element,Data_Value,NAME
0,JA000047899,2007-12-03,TMAX,158,MUROTOMISAKI
1,JA000047899,2006-09-01,TMIN,223,MUROTOMISAKI
2,JA000047899,2014-01-17,TMAX,108,MUROTOMISAKI
3,JA000047899,2008-04-15,TMIN,123,MUROTOMISAKI
4,JA000047899,2015-03-01,TMAX,124,MUROTOMISAKI
5,JA000047899,2006-09-01,TMAX,267,MUROTOMISAKI
6,JA000047899,2014-04-21,TMAX,173,MUROTOMISAKI
7,JA000047899,2014-04-21,TMIN,124,MUROTOMISAKI
8,JA000047899,2012-11-09,TMAX,196,MUROTOMISAKI
9,JA000047899,2006-05-27,TMIN,150,MUROTOMISAKI


In [24]:
df_total[df_total['ID'].str.contains(r'^CHM')]
df_total.loc[:,'Date'] = pd.to_datetime(df_total['Date'])

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
